In [9]:
pip install spacy textblob

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
     - ------------------------------------- 30.7/636.8 kB 1.4 MB/s eta 0:00:01
     --------------- ---------------------- 256.0/636.8 kB 3.2 MB/s eta 0:00:01
     -------------------------------------  634.9/636.8 kB 5.7 MB/s eta 0:00:01
     -------------------------------------- 636.8/636.8 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----------- ---------------------------- 0.4/1.5 MB 8.7 MB/s eta 0:00:01
     ------------------------ --------------- 0.9/1.5 MB 9.9 MB/s eta 0:00:01
     ---------------------------------------  1.5/1.5 MB 10.6 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 10.6 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
pip install seaborn

   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
   ---------------------------------------- 294.6/294.6 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

     ---------------------------------------- 0.0/159.4 kB ? eta -:--:--
     ---------------------- ---------------- 92.2/159.4 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 159.4/159.4 kB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   -- ------------------------------------- 0.4/7.6 MB 8.5 MB/s eta 0:00:01
   ---- ----------------------------------- 0.9/7.6 MB 9.6 MB/s eta 0:00:01
   ------- -------------------------------- 1.5/7.6 MB 10.5 MB/s eta 0:00:01
   ---------- ----------------------------- 2.1/7.6 MB 11.0 MB/s eta 0:00:01
   ------------- -------------------------- 2.7/7.6 MB 10.6 MB/s eta 0:00:01
   ---------------- ----------------------- 3.2/7.6 MB 10.9 MB/s eta 0:00:01
   ------------------- -------------------- 3.7/7.6 MB 10.9 MB/s eta 0:00:01
   ---------------------- ----------------- 4.3/7.6 MB 11.0 MB/s eta 0:00:01
   ------------------------- -------------- 4.9/7.6 MB 11.1 MB/s eta 0:00:01
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, SpatialDropout1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from textblob import TextBlob
from tensorflow.keras.layers import Bidirectional, Dropout

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the dataset
df = pd.read_csv('cyberbullying_tweets.csv')

# Drop unnecessary columns (e.g., age, gender, religion, ethnicity)
df = df[['tweet_text', 'cyberbullying_type']]

# Split the data into training and testing sets
train_texts, test_texts, y_train, y_test = train_test_split(df['tweet_text'], df['cyberbullying_type'], test_size=0.2, random_state=42)

def classify_severity(tweet):
    # Perform sentiment analysis using TextBlob
    sentiment = TextBlob(tweet).sentiment

    # Use spaCy for tokenization and lemmatization
    doc = nlp(tweet.lower())
    tokens = [token.lemma_ for token in doc]

    # Custom rules for severity classification
    high_severity_keywords = ['hate', 'threat', 'violence']
    medium_severity_keywords = ['insult', 'harassment']

    if any(keyword in tokens for keyword in high_severity_keywords) or sentiment.polarity < -0.5:
        return 'High'
    elif any(keyword in tokens for keyword in medium_severity_keywords) or sentiment.polarity < 0:
        return 'Medium'
    else:
        return 'Low'

# Apply the classification function to the training and testing sets
y_train_pred = [classify_severity(tweet) for tweet in train_texts]
y_test_pred = [classify_severity(tweet) for tweet in test_texts]

# Preprocess the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_pred)
y_test_encoded = label_encoder.transform(y_test_pred)

# Tokenize and pad the text data for GRU model
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

x_train = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
x_test = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Build the GRU model
embedding_dim = 200
gru_units = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.5))
model.add(Bidirectional(GRU(gru_units, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
model.add(Bidirectional(GRU(gru_units, dropout=0.5, recurrent_dropout=0.5)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Implement a learning rate scheduler
def lr_scheduler(epoch, lr):
    return lr * 0.9  # Adjust the factor as needed

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
epochs = 20  # Increase the number of epochs
batch_size = 64  # Adjust the batch size

history = model.fit(x_train, y_train_encoded, epochs=epochs, batch_size=batch_size,
                    validation_split=0.1, callbacks=[early_stopping, lr_scheduler_callback])

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test_encoded)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')



Epoch 1/20


537/537 [==============================] - 846s 2s/step - loss: 1.3145 - accuracy: 0.6756 - val_loss: 0.6168 - val_accuracy: 0.7909 - lr: 9.0000e-04
Epoch 2/20
537/537 [==============================] - 894s 2s/step - loss: 0.4809 - accuracy: 0.8318 - val_loss: 0.3948 - val_accuracy: 0.8546 - lr: 8.1000e-04
Epoch 3/20
537/537 [==============================] - 923s 2s/step - loss: 0.3473 - accuracy: 0.8772 - val_loss: 0.3336 - val_accuracy: 0.8831 - lr: 7.2900e-04
Epoch 4/20
537/537 [==============================] - 911s 2s/step - loss: 0.2848 - accuracy: 0.9047 - val_loss: 0.3289 - val_accuracy: 0.8868 - lr: 6.5610e-04
Epoch 5/20
537/537 [==============================] - 919s 2s/step - loss: 0.2479 - accuracy: 0.9192 - val_loss: 0.3268 - val_accuracy: 0.8842 - lr: 5.9049e-04
Epoch 6/20
537/537 [==============================] - 929s 2s/step - loss: 0.2201 - accuracy: 0.9307 - val_loss: 0.3308 - val_accuracy: 0.8902 - lr: 5.3144e-04
Epoch 7/20
537/537 [================